# Haleema Sadia


# Task 10: ML Facial recognition to detect mood and suggest songs accordingly

# Lgmvip Data Science Internship july

## Dataset Link : https://www.kaggle.com/datasets/msambare/fer2013 

In [ ]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
